In [1]:
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [2]:
# Соединение с БД
engine = create_engine('sqlite:///:memory:', echo=True)

In [3]:
# Декларативное создание таблицы, класса и отображения за один раз
Base = declarative_base()
# Создание таблицы в БД
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
# Определение класса для отображения в таблицу   
    def __init__(self, name, fullname, password):
        self.name = name
        self.fullname = fullname
        self.password = password
    def __repr__(self):
        return "<User('%s','%s', '%s')>" % (self.name, self.fullname, self.password)

In [4]:
# Создание таблицы
Base.metadata.create_all(engine)

2020-03-15 00:12:22,597 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-15 00:12:22,612 INFO sqlalchemy.engine.base.Engine ()
2020-03-15 00:12:22,612 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-15 00:12:22,612 INFO sqlalchemy.engine.base.Engine ()
2020-03-15 00:12:22,612 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-03-15 00:12:22,620 INFO sqlalchemy.engine.base.Engine ()
2020-03-15 00:12:22,622 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-03-15 00:12:22,624 INFO sqlalchemy.engine.base.Engine ()
2020-03-15 00:12:22,627 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2020-03-15 00:12:22,628 INFO sqlalchemy.engine.base.Engine ()
2020-03-15 00:12:22,630 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
# Создание сессии
Session = sessionmaker(bind = engine)
session = Session()

In [6]:
# Добавление новых пользователей. Вариант 1
user1 = User("Alexander", "Alexander Smith", "123*")
session.add(user1)
session.commit()

2020-03-15 00:12:22,795 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-15 00:12:22,795 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2020-03-15 00:12:22,795 INFO sqlalchemy.engine.base.Engine ('Alexander', 'Alexander Smith', '123*')
2020-03-15 00:12:22,811 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
# Добавление новых пользователей. Вариант 2
session.add_all([
   User(name = "Daniel", fullname = "Daniel Johnson", password = "456*"), 
   User(name = "Logan", fullname = "Logan Miller", password = "789*"), 
   User(name = "David", fullname = "David Taylor", password = "000*")]
)
session.commit()

2020-03-15 00:12:22,886 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-15 00:12:22,887 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2020-03-15 00:12:22,888 INFO sqlalchemy.engine.base.Engine ('Daniel', 'Daniel Johnson', '456*')
2020-03-15 00:12:22,889 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2020-03-15 00:12:22,889 INFO sqlalchemy.engine.base.Engine ('Logan', 'Logan Miller', '789*')
2020-03-15 00:12:22,890 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2020-03-15 00:12:22,891 INFO sqlalchemy.engine.base.Engine ('David', 'David Taylor', '000*')
2020-03-15 00:12:22,893 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
# Запросы. Вариант 1
for instance in session.query(User).order_by(User.id): 
    print (instance.name, instance.fullname)

2020-03-15 00:12:22,950 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-15 00:12:22,951 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2020-03-15 00:12:22,952 INFO sqlalchemy.engine.base.Engine ()
Alexander Alexander Smith
Daniel Daniel Johnson
Logan Logan Miller
David David Taylor


In [9]:
# Запросы. Вариант 2
for name, fullname in session.query(User.name, User.fullname): 
    print (name, fullname)

2020-03-15 00:12:23,017 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2020-03-15 00:12:23,018 INFO sqlalchemy.engine.base.Engine ()
Alexander Alexander Smith
Daniel Daniel Johnson
Logan Logan Miller
David David Taylor


In [10]:
# Запросы. Вариант 3
for row in session.query(User, User.name).all(): 
    print (row.User, row.name)

2020-03-15 00:12:23,096 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2020-03-15 00:12:23,112 INFO sqlalchemy.engine.base.Engine ()
<User('Alexander','Alexander Smith', '123*')> Alexander
<User('Daniel','Daniel Johnson', '456*')> Daniel
<User('Logan','Logan Miller', '789*')> Logan
<User('David','David Taylor', '000*')> David


In [11]:
# Запросы. Срезы (LIMIT)
for instance in session.query(User).order_by(User.id) [0:2]: 
    print (instance.name, instance.fullname)

2020-03-15 00:12:23,184 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2020-03-15 00:12:23,185 INFO sqlalchemy.engine.base.Engine (2, 0)
Alexander Alexander Smith
Daniel Daniel Johnson


In [12]:
# Запросы. Фильтрация результатов
for instance in session.query(User).filter(User.fullname=='Daniel Johnson'): 
    print (instance.name, instance.fullname)

2020-03-15 00:12:23,261 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.fullname = ?
2020-03-15 00:12:23,262 INFO sqlalchemy.engine.base.Engine ('Daniel Johnson',)
Daniel Daniel Johnson


In [13]:
# Запросы. Подобно
for instance in session.query(User).filter(User.password.like("0%")): 
    print (instance.name, instance.fullname, instance.password)

2020-03-15 00:12:23,365 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.password LIKE ?
2020-03-15 00:12:23,366 INFO sqlalchemy.engine.base.Engine ('0%',)
David David Taylor 000*


In [14]:
# Запросы. Метод IN
for instance in session.query(User).filter(User.password.in_(['123*','456*'])):
    print (instance.name, instance.fullname, instance.password)

2020-03-15 00:12:23,438 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.password IN (?, ?)
2020-03-15 00:12:23,439 INFO sqlalchemy.engine.base.Engine ('123*', '456*')
Alexander Alexander Smith 123*
Daniel Daniel Johnson 456*


In [15]:
# Запросы. Методы AND / OR
for instance in session.query(User).filter(((User.name =='Daniel') and (User.password.in_(['123*','456*'])))):
    print (instance.name, instance.fullname, instance.password)

2020-03-15 00:12:23,506 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2020-03-15 00:12:23,507 INFO sqlalchemy.engine.base.Engine ('Daniel',)
Daniel Daniel Johnson 456*
